In [1]:
import yaml
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
from Explanations_Models.DT_LIME.LIME import LIME
from Explanations_Models.DT_LIME.sampling_methods import Uniform_Sampler
import json
yaml.add_constructor("!join", join)
config = yaml.load(open("config_envs/cartpole.yml"), Loader= yaml.FullLoader)
config.update(yaml.load(open("config_explanations/cartpole.yml"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights()


device:  cpu


UnboundLocalError: local variable 'PATH_critic' referenced before assignment

In [2]:

PercentCorrect = []
ExecutionMSE = []
ExecutionDiff = []
TDepth = []
TBreadth = []
top_splits = []
for i in range(config["metric_hyperparameters"]["tree_execution_samples"]):
    limemod = LIME(config, Runner)
    limemod.train()
    PercentCorrect.append(limemod.percent_Correct())
    ADmse, CountDiff = limemod.absolute_distance()
    ExecutionMSE.append(ADmse)
    ExecutionDiff.append(CountDiff)

    depth, breadth = limemod.surr_model.depth_breadth()
    TDepth.append(depth)
    TBreadth.append(breadth)
    top_splits.append(config["picture"]["labels"][limemod.surr_model.get_top_split()])



#limemod.surr_model.display_tree()


0.825 0.030956296 0 7 8 angle_vel


c:\Users\blav1\anaconda3\envs\XAI_RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\DT_LIME\sampling_methods.py:47: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  return torch.tensor(samples, dtype = torch.float32)[samples_indicies]
c:\Users\blav1\anaconda3\envs\XAI_RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\b

In [3]:
limemod.surr_model.depth_breadth()

(7, 8)

In [5]:
config["picture"]["labels"][limemod.surr_model.get_top_split()]

'angle_vel'

In [4]:
sampler = Uniform_Sampler(config, Runner = None)

NameError: name 'Uniform_Sampler' is not defined